In [1]:
import json
import os
import pandas as pd
import re
import time
import tweepy

from datetime import datetime, date
from textblob import TextBlob
from HTMLParser import HTMLParser

In [2]:
consumer_key = '4rYHwbRjk7wRxIMKGIKhhmWzD'
consumer_secret = 'TNuu7wNi95MIf6gKtSVHzizDAkbuqqAfAr0BD99cgChZZbxBoc'

access_token = '2337278024-uMnCA5bHvvefkdk3xjQbxNrv3linA2P7aebAnR2'
access_token_secret = 'bwA1nvr0o3M3hb7ay7xxqPOUmeyKfssY1DS3Ck6UNvPCB'

In [3]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split())

def parse_tweets(tweets):
    sentiment = pd.Series()
    subjectivity = pd.Series()
    retweets = pd.Series()
    likes = pd.Series()
    time = pd.Series()
    text = pd.Series()
    tweet_id = pd.Series()

    for tweet_1 in tweets:
        if hasattr(tweet_1, 'retweeted_status'):
            tweet = tweet_1.retweeted_status
        else:
            tweet = tweet_1
        
        analysis = TextBlob(clean_tweet(tweet.text))
        # saving sentiment of tweet
        sentiment = sentiment.append(pd.Series(analysis.sentiment.polarity))
        subjectivity = subjectivity.append(pd.Series(analysis.sentiment.subjectivity))
        
        retweets = retweets.append(pd.Series(tweet.retweet_count))
        likes = likes.append(pd.Series(tweet.favorite_count))
        time = time.append(pd.Series(tweet.created_at))
        raw_text = tweet.text.replace(u'\\', u'/')
        text = text.append(pd.Series(raw_text.encode('utf-8').decode('unicode_escape').encode('ascii','ignore')))
        tweet_id = tweet_id.append(pd.Series(tweet.id))
        
    metrics = pd.DataFrame()
    metrics['sentiment'] = sentiment
    metrics['positive_dummy'] = metrics.apply(lambda row: int(row['sentiment'] > 0), axis=1)
    metrics['neutral_dummy'] = metrics.apply(lambda row: int(row['sentiment'] == 0), axis=1)
    metrics['negative_dummy'] = metrics.apply(lambda row: int(row['sentiment'] < 0), axis=1)
    metrics['subjectivity'] = subjectivity
    metrics['retweets'] = retweets
    metrics['likes'] = likes
    metrics['timestamp'] = time
    metrics['text'] = text
    metrics['id'] = tweet_id
    
    metrics = metrics.replace("\\", "/")
    return metrics

In [ ]:
past_scraped_tweets = pd.DataFrame()
for num in range(1, 13):
    archive = pd.read_csv('archive/archive_{0}.csv'.format(num))
    past_scraped_tweets = past_scraped_tweets.append(archive).drop_duplicates().sort_values(by='timestamp', 
                                                                                            ascending=False)
    print 'archive #{0} added...'.format(num)

In [32]:
x = pd.read_csv('archive/archive_12.csv')

In [31]:
past_scraped_tweets = past_scraped_tweets.append(all_tweets).drop_duplicates()

In [13]:
#past_scraped_tweets = past_scraped_tweets.append(pd.read_csv('archive/archive_11.csv')).drop_duplicates()

past_scraped_tweets = past_scraped_tweets.sort_values(by='timestamp', ascending=False)

In [9]:
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True)

In [35]:
# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
#sinceId = float(past_scraped_tweets.head(1)['id'])
sinceId = None
# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -1L

tweetCount = 0

In [36]:
searchQuery = 'trump'  # this is what we're searching for
maxTweets = 10000 #1000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
fName = 'tweets_archive.csv' # We'll store the tweets in a text file.

all_tweets = pd.DataFrame()
raw_tweets = []

print("Downloading max {0} tweets".format(maxTweets))
with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            #### search
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry)
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since_id=sinceId)
            else:
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since="2017-03-13")
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets:
                print("No more tweets found")
                break
            raw_tweets.append(new_tweets)
            print len(raw_tweets)
            #### parse extracted data
            parsed_tweets = parse_tweets(new_tweets)
            parsed_tweets = parsed_tweets.drop_duplicates()
            
            tweetCount += len(parsed_tweets)
            
            all_tweets = all_tweets.append(parsed_tweets)
            all_tweets = all_tweets.drop_duplicates()
            print("Downloaded {0} tweets".format(tweetCount))
            max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break

print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

1
Downloaded 95 tweets
2
Downloaded 192 tweets
3
Downloaded 288 tweets
4
Downloaded 386 tweets
5
Downloaded 481 tweets
6
Downloaded 577 tweets
7
Downloaded 672 tweets
8
Downloaded 766 tweets
9
Downloaded 860 tweets
10
Downloaded 952 tweets
11
Downloaded 1046 tweets
12
Downloaded 1140 tweets
13
Downloaded 1235 tweets
14
Downloaded 1329 tweets
15
Downloaded 1421 tweets
16
Downloaded 1514 tweets
17
Downloaded 1610 tweets
18
Downloaded 1706 tweets
19
Downloaded 1804 tweets
20
Downloaded 1899 tweets
21
Downloaded 1994 tweets
22
Downloaded 2089 tweets
23
Downloaded 2184 tweets
24
Downloaded 2278 tweets
25
Downloaded 2374 tweets
26
Downloaded 2469 tweets
27
Downloaded 2564 tweets
28
Downloaded 2662 tweets
29
Downloaded 2760 tweets
30
Downloaded 2855 tweets
31
Downloaded 2952 tweets
32
Downloaded 3051 tweets
33
Downloaded 3148 tweets
34
Downloaded 3243 tweets
35
Downloaded 3337 tweets
36
Downloaded 3433 tweets
37
Downloaded 3527 tweets
38
Downloaded 3621 tweets
39
Downloaded 3716 tweets
40
Dow

In [20]:
for index, tweet in parsed_tweets.sample(5).iterrows():
    analysis = TextBlob(tweet.text)
    print 'text: ', analysis
    print 'polarity: ', analysis.sentiment.polarity
    print 'subjectivity: ', analysis.sentiment.subjectivity, '\n'

text:  Key Words: Newly redesigned HuffPost makes a pitch to Trump voters.. Related Articles: https://t.co/EGnv4jXFXt
polarity:  0.0454545454545
subjectivity:  0.618181818182 

text:  Dear #MAGA boomers:  this will destroy millions of your lives and even bankrupt some of you.  Time to get angry.  T https://t.co/Sy43CDfx5a
polarity:  -0.35
subjectivity:  0.5 

text:  Trump's first 100 days in office also happens to be 100 of Twitter's least funniest days.
polarity:  -0.025
subjectivity:  0.366666666667 

text:  If you believe in the Easter Bunny, then you might believe the Trump #taxreform pays for itself. For the rest of us, it is a bigly disaster.
polarity:  0.0
subjectivity:  0.1 

text:  @YoungDems4Trump I'm sure Trump "means" business, but setting up a helpline is not exactly a big ticket item...
polarity:  0.25
subjectivity:  0.494444444444 



In [37]:
all_tweets

,sentiment,positive_dummy,neutral_dummy,negative_dummy,subjectivity,retweets,likes,timestamp,text,id
0,0.000000,0,1,0,0.000000,50,72,2017-04-27 16:55:05,Donald Trumps assault on the nations courts is...,857639246890500096
0,0.133333,1,0,0,0.916667,0,0,2017-04-27 18:52:09,"10/ That is: you may think NAFTA is great, ter...",857668707195203584
0,-0.412500,0,0,1,0.395833,0,0,2017-04-27 18:52:09,@ShawnSRocks @thehill Trump's lining his pocke...,857668706943500288
0,0.000000,0,1,0,0.000000,93,146,2017-04-27 16:59:25,Peak Trump with Marci on the lemon business in...,857640338596474882
0,-0.212500,0,0,1,0.458333,1,1,2017-04-27 18:48:27,12 Trump tweets that have aged the worst in hi...,857667776651165696
0,0.200000,1,0,0,0.500000,315,927,2017-04-27 15:23:00,"The ACLU today, tomorrow, and for the rest of...",857616076066045952
0,0.000000,0,1,0,0.000000,37,43,2017-04-27 18:32:33,"Donald Trump: ""Yo le voy a hablar de Corea del...",857663775964618752
0,-0.066667,0,0,1,0.266667,680,502,2017-04-26 01:31:44,Holy shit. The U.N. notified us on Feb. 2nd th...,857044489365598213
0,0.000000,0,1,0,0.000000,0,0,2017-04-27 18:52:09,Las 3 grandes promesas que Donald Trump no pud...,857668706645790722
0,0.000000,0,1,0,0.000000,331,326,2017-04-26 23:43:14,USA: Trump asked Merkel ten times if US could ...,857379573935128576


In [31]:
for tweet in raw_tweets:
    if tweet.place:
        print tweet.place.full_name

AttributeError: 'SearchResults' object has no attribute 'place'

In [25]:
len(new_tweets)

100

In [27]:
len(all_tweets)

414125

In [122]:
sinceId = float(all_tweets.sort_values(by='timestamp', ascending=False)['id'].head(1))

In [ ]:
searchQuery = 'trump'  # this is what we're searching for
maxTweets = 10000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
fName = 'tweets_archive.csv' # We'll store the tweets in a text file.

# all_tweets = pd.DataFrame()

print("Downloading max {0} tweets".format(maxTweets))
with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            #### search
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry)
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since_id=sinceId)
            else:
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since="2017-03-13")
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets:
                print("No more tweets found")
                break
            
            #### parse extracted data
            parsed_tweets = parse_tweets(new_tweets)
            parsed_tweets = parsed_tweets.drop_duplicates()
            
            tweetCount += len(parsed_tweets)
            
            all_tweets = all_tweets.append(parsed_tweets)
            all_tweets = all_tweets.drop_duplicates()
            print("Downloaded {0} tweets".format(tweetCount))
            max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break

print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

In [ ]:
print '% positive: {0}%'.format(sum(all_tweets.positive_dummy)/float(len(all_tweets)) * 100)
print '# of total tweets: {0}'.format(len(all_tweets))

In [ ]:
print len(all_tweets_20170320.drop_duplicates())
print len(all_tweets_20170320.append(parsed_tweets).drop_duplicates())

In [ ]:
auth = tweepy.auth.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
search_results = api.search(q="trump",
                            since="2017-03-10",
                            until="2017-03-11",
                            count=100000000)

In [ ]:
search_results_1 = api.search(q='trump',
                             since='2017-03-13',
                             until='2017-03-14',
                             count=1000000)

In [ ]:
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True)

In [ ]:
searchQuery = 'trump'  # this is what we're searching for
maxTweets = 10000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
fName = 'tweets_archive.csv' # We'll store the tweets in a text file.

# all_tweets = pd.DataFrame()

print("Downloading max {0} tweets".format(maxTweets))
with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            #### search
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry)
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since_id=sinceId)
            else:
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since="2017-03-13")
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets:
                print("No more tweets found")
                break

In [ ]:
try:
    tweets = []
    # parsing tweets one by one
    for tweet in search_results:
        # saving text of tweet
        if hasattr(tweet, 'retweeted_status'):
            parsed_tweet = extract_tweet_data(tweet.retweeted_status)
        else:
            parsed_tweet = extract_tweet_data(tweet)
 
        # appending parsed tweet to tweets list
        if parsed_tweet not in tweets:
            tweets.append(parsed_tweet)
        else:
            tweets.append(parsed_tweet)
        
except tweepy.TweepError as e:
    # print error (if any)
    print("Error : " + str(e))
    


In [ ]:
tweet_1 = all_tweets.iloc[0,:]
tweet_2 = all_tweets.iloc[1,:]

print tweet_1.timestamp.date() < date(2017, 3, 17)
print tweet_1.timestamp.date()

In [90]:
import csv, io
test = pd.DataFrame()
with open("archive/archive_10.csv", 'r') as f:
    with io.open("archive/archive_10_fixed.csv", "w", newline='') as writing_file:
        writer = csv.writer(writing_file)
        reader = csv.reader(f)
        linenumber = 1
        for row in reader:
            linenumber += 1

        
        
        
#        for row in f:
##            writing_file.write(unicode(row, 'utf-8'))
 #          linenumber += 1
            #try:
            #    writer.writerow(row)
            #except Exception as e:
            #    print (("Error line %d: %s" % (linenumber, str(type(e)))))
            #    continue
            #writer.writerow(un)

Error: new-line character seen in unquoted field - do you need to open the file in universal-newline mode?

In [28]:
all_tweets['timestamp date'] = all_tweets.apply(lambda row: row['timestamp'].date(), axis=1)

In [29]:
all_tweets['text'] = all_tweets.apply(lambda row: row['text'].encode('utf-8').decode('unicode_escape').encode('ascii','ignore'), axis=1)

all_tweets['text'] = all_tweets.apply(lambda row: row['text'].replace('\n', ' '), axis=1)

In [125]:
all_tweets

,sentiment,positive_dummy,neutral_dummy,negative_dummy,subjectivity,retweets,likes,timestamp,text,id,timestamp date
0,0.000000,0,1,0,0.312500,6,3,2017-04-18 00:29:59,Entire Western World Helped Obama #WIRETAP Tru...,854129850398851073,2017-04-18
0,-0.062500,0,0,1,0.687500,556,1190,2017-04-17 22:31:11,Trump's unpopularity is affecting other Republ...,854099949872480257,2017-04-17
0,0.252381,1,0,0,0.523810,0,0,2017-04-18 00:33:47,A Montana special election nobody is following...,854130803831300097,2017-04-18
0,0.000000,0,1,0,0.000000,3783,4839,2017-04-17 15:18:25,VOTE TOMORROW for Jon Ossoff (@ossoff) in Geor...,853991040889606145,2017-04-17
0,0.000000,0,1,0,0.066667,5,2,2017-04-18 00:28:44,Sounds like @DeanHeller has a lot of mind chan...,854129534496460800,2017-04-18
0,0.112121,1,0,0,0.384848,287,544,2017-04-17 23:00:00,NEW VIDEO: Could Trump really have been moroni...,854107205473357829,2017-04-17
0,0.150000,1,0,0,0.133333,3,20,2017-04-18 00:30:19,Fun fact: That was also the last Trump Easter ...,854129932695482369,2017-04-18
0,0.000000,0,1,0,0.000000,150,245,2017-02-22 02:48:34,Representative @MaxineWaters on @MSNBC: Trump ...,834233393286090753,2017-02-22
0,-0.100000,0,0,1,0.500000,295,766,2017-04-17 23:26:40,If the Democrats are marching on Trump's tax r...,854113913356636160,2017-04-17
0,0.000000,0,1,0,0.000000,490,1853,2017-04-17 21:49:59,Pumped to support @americafirstpol as they pus...,854089583990108160,2017-04-17


In [30]:
all_tweets.to_csv('archive/archive_12.csv', index=False)